In [373]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from __future__ import print_function

from datetime import date
import datetime
#imports

# Ideas

Raise error when len does not match before and after merging datasets (means that a player wasn't found -- likely because he is a junior or has two last names, etc.)

# To Do

1. Address last name issue
2. Get pitcher xba onto
3. Scrape data instead of downloading CSVs

ON avoiding duplicate names --- maybe could change xba column in pitchers df before I merge, that way any batters with the name Luis Garcia would not have the column and be left out

In [374]:
batters_df = pd.read_csv('../../data/individual_batting_stats.csv')
pitchers_df = pd.read_csv('../../data/individual_pitching_stats.csv')
misc_batters = pd.read_csv('../../data/batters_teams.csv')
misc_pitchers = pd.read_csv('../../data/pitchers_teams.csv')

In [375]:
len(pitchers_df)

506

In [376]:
y = batters_df[batters_df['last_name'].str.contains(" ")]
batters_df['last_name'] = [i.split(' ')[0] for i in batters_df['last_name']]
#The teams df does not have Jr., III for example listed on the names so I will cut them off before that so they match

In [377]:
batters_df.loc[batters_df['last_name'] == 'Cruz']

,last_name,first_name,player_id,year,b_k_percent,b_bb_percent,batting_avg,xba,xslg,woba,xwoba,xobp,xiso,xbadiff,xslgdiff,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,Unnamed: 18
5,Cruz,Nelson,443558,2022,18.8,9.4,0.219,0.273,0.495,0.272,0.356,0.343,0.222,-0.054,-0.184,90.2,8.4,10.8,NaN


In [378]:
len(batters_df)

425

# Probable Pitchers

In [379]:
d = datetime.datetime.now()
today = d.strftime("%Y-%m-%d")
#Get today's date

In [380]:
url = f'https://www.mlb.com/probable-pitchers/{today}'
#URL of where I will get the pitchers' names

In [381]:
res = requests.get(url)
soup = BeautifulSoup(res.content)
#Making request to the site and storing it in soup

In [382]:
# pitchers = soup.find_all('div', {'class': 'probable-pitchers__pitcher-name'})

In [383]:
pitchers = soup.find_all('div', {'class': 'probable-pitchers__pitcher-name'})
probable_pitchers = []
for i in pitchers:
    probable_pitchers.append(str(i))
#Finding all of the pitchers' names within that <a> tag
#Using for loop to convert the beautiful soup elements in the list to strings so I can get only the pitcher's name from there

In [384]:
probable_pitchers = [i.split('\n')[1] for i in probable_pitchers]
#Getting only the pitchers' names

In [385]:
for i in range(len(probable_pitchers)):
    if 'TBD' in probable_pitchers[i]:
        probable_pitchers[i] = '>TBD</a>'

In [386]:
probable_pitchers

['<a class="probable-pitchers__pitcher-name-link" href="/player/carlos-rodon-607074">Carlos Rodon</a>',
 '>TBD</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/austin-gomber-596295">Austin Gomber</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/aaron-sanchez-592717">Aaron Sanchez</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/kyle-bradish-680694">Kyle Bradish</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/garrett-whitlock-676477">Garrett Whitlock</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/shane-bieber-669456">Shane Bieber</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/alex-faedo-656412">Alex Faedo</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/bailey-falter-663559">Bailey Falter</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/carlos-carrasco-471911">Carlos Carrasco</a>',
 '<a class="probable-pitchers__pitc

In [387]:
probable_pitchers = [i.split('</a')[0] for i in probable_pitchers] 
#Getting only the pitchers' names part 2

In [388]:
probable_pitchers = [i.split('>')[1] for i in probable_pitchers] 

In [389]:
probable_pitchers

['Carlos Rodon',
 'TBD',
 'Austin Gomber',
 'Aaron Sanchez',
 'Kyle Bradish',
 'Garrett Whitlock',
 'Shane Bieber',
 'Alex Faedo',
 'Bailey Falter',
 'Carlos Carrasco',
 'Jameson Taillon',
 'Jeffrey Springs',
 'Trevor Rogers',
 'Ian Anderson',
 'Brad Keller',
 'Bailey Ober',
 'Brandon Woodruff',
 'Dakota Hudson',
 'Alek Manoah',
 'Chase Silseth',
 'Ryan Pepiot',
 'Madison Bumgarner',
 'Jose Quintana',
 'Sean Manaea',
 'Justin Verlander',
 'Chris Flexen',
 'Jon Gray',
 'Cole Irvin']

In [390]:
probable_pitchers = [i.split() for i in probable_pitchers]
#Splitting name into first and last name columns

In [391]:
probable_pitchers

[['Carlos', 'Rodon'],
 ['TBD'],
 ['Austin', 'Gomber'],
 ['Aaron', 'Sanchez'],
 ['Kyle', 'Bradish'],
 ['Garrett', 'Whitlock'],
 ['Shane', 'Bieber'],
 ['Alex', 'Faedo'],
 ['Bailey', 'Falter'],
 ['Carlos', 'Carrasco'],
 ['Jameson', 'Taillon'],
 ['Jeffrey', 'Springs'],
 ['Trevor', 'Rogers'],
 ['Ian', 'Anderson'],
 ['Brad', 'Keller'],
 ['Bailey', 'Ober'],
 ['Brandon', 'Woodruff'],
 ['Dakota', 'Hudson'],
 ['Alek', 'Manoah'],
 ['Chase', 'Silseth'],
 ['Ryan', 'Pepiot'],
 ['Madison', 'Bumgarner'],
 ['Jose', 'Quintana'],
 ['Sean', 'Manaea'],
 ['Justin', 'Verlander'],
 ['Chris', 'Flexen'],
 ['Jon', 'Gray'],
 ['Cole', 'Irvin']]

In [392]:
# probable_pitchers.insert(1, ['Roansy', 'Contreras'])
#Temporary

In [393]:
df_probable_pitchers = pd.DataFrame(probable_pitchers)
#Setting list as Pandas DF

In [394]:
df_probable_pitchers = df_probable_pitchers.rename(columns={0: "first_name", 1: "last_name"})
#Renaming column

# Get Pitchers' Teams

In [395]:
new_teams = soup.find_all('span', {'class':['probable-pitchers__team-name probable-pitchers__team-name--away', 'probable-pitchers__team-name probable-pitchers__team-name--home']})
teams_of_probable_pitchers = []
for i in new_teams:
    teams_of_probable_pitchers.append(str(i))
#Finding all of the pitchers' names within that <a> tag
#Using for loop to convert the beautiful soup elements in the list to strings so I can get only the pitcher's name from there

In [396]:
teams_of_probable_pitchers = [x.strip(' ') for x in teams_of_probable_pitchers]
#Getting only the pitchers' names

In [397]:
teams_of_probable_pitchers = [i.split('>\n')[1] for i in teams_of_probable_pitchers] 
#Getting only the pitchers' names

In [398]:
teams_of_probable_pitchers = teams_of_probable_pitchers = [x.strip('                             ') for x in teams_of_probable_pitchers]
#Stripping white space

In [399]:
teams_of_probable_pitchers = [i.split('\n')[0] for i in teams_of_probable_pitchers] 

In [400]:
len(teams_of_probable_pitchers)

28

In [401]:
df_probable_pitchers['team'] = teams_of_probable_pitchers
#Putting team on this way, now we just need to change it to the abbreviation

In [402]:
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Astros', 'HOU', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Nationals', 'WAS', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Giants', 'SF', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Blue Jays', 'TOR', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Angels', 'LAA', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'White Sox', 'CWS', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Guardians', 'CLE', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Athletics', 'OAK', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Twins', 'MIN', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Rays', 'TB', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Mariners', 'SEA', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Marlins', 'MIA', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Padres', 'SD', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Pirates', 'PIT', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Red Sox', 'BOS', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Orioles', 'BAL', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Braves', 'ATL', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Rangers', 'TEX', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Phillies', 'PHI', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Mets', 'NYM', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Yankees', 'NYY', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Royals', 'KC', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Cubs', 'CHC', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Brewers', 'MIL', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Reds', 'CIN', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Rockies', 'COL', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Tigers', 'DET', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Dodgers', 'LAD', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'D-backs', 'ARI', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Cardinals', 'STL', df_probable_pitchers['team'])
#Changing team nickname to abbreviation so it will match the other DF -- Will clean this up in the future

In [403]:
misc_pitchers = misc_pitchers[['Player', 'Team']]
#All we need from this is their names and the team, because we will merge it with the other df to get the name on there

In [404]:
misc_pitchers[['first_name', 'last_name']] = misc_pitchers['Player'].str.split(' ', 1, expand=True)
#Separating 'Player' column into two columns for first and last to match probable_pitchers df

In [405]:
misc_pitchers = misc_pitchers.drop(columns='Player')
#We don't need this full name column anymore

In [406]:
misc_pitchers = misc_pitchers.rename(columns={"Team": "team"})
#Make this lowercase so it matches team column in probable pitchers

In [407]:
misc_pitchers = misc_pitchers[['last_name', 'first_name', 'team']]
#reordering columns

In [408]:
misc_pitchers.loc[misc_pitchers['last_name'] == 'Garcia']

,last_name,first_name,team
81,Garcia,Luis,HOU
205,Garcia,Yimi,TOR
346,Garcia,Rony,DET
409,Garcia,Jarlin,SF
437,Garcia,Luis,SD


# Get Batters' Teams

In [409]:
misc_batters = misc_batters[['Player', 'Team']]
#All we need from this is their names and the team, because we will merge it with the other df to get the name on there

In [410]:
misc_batters[['first_name', 'last_name']] = misc_batters['Player'].str.split(' ', 1, expand=True)
#Separating 'Player' column into two columns for first and last to match probable_pitchers df

In [411]:
misc_batters = misc_batters.drop(columns='Player')
#We don't need this full name column anymore

In [412]:
misc_batters = misc_batters[['last_name', 'first_name', 'Team']]
#Reordering columns

In [413]:
pitchers_df['first_name'] = pitchers_df['first_name'].str.strip()
batters_df['first_name'] = batters_df['first_name'].str.strip()
#Removing empty space in these column entries for example this makes ' Cabrera' => 'Cabrera'

batters_df and pitchers_df front names all have a space in front of them and that's why merge is not working

# Attaching according Team Name to each player in all DFs

The datasets on Baseball Savant do not include the team name with they players, so I will add them in

In [414]:
misc_pitchers.loc[misc_pitchers['last_name'] == 'Garcia']

,last_name,first_name,team
81,Garcia,Luis,HOU
205,Garcia,Yimi,TOR
346,Garcia,Rony,DET
409,Garcia,Jarlin,SF
437,Garcia,Luis,SD


In [415]:
misc_pitchers = misc_pitchers.drop(misc_pitchers[misc_pitchers.last_name == 'Garcia'].index & misc_pitchers[misc_pitchers.first_name == 'Luis'].index & misc_pitchers[misc_pitchers.team == 'SD'].index)

C:\Users\User\AppData\Local\Temp/ipykernel_13300/3128343429.py:1: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  misc_pitchers = misc_pitchers.drop(misc_pitchers[misc_pitchers.last_name == 'Garcia'].index & misc_pitchers[misc_pitchers.first_name == 'Luis'].index & misc_pitchers[misc_pitchers.team == 'SD'].index)


In [416]:
misc_pitchers.loc[misc_pitchers['last_name'] == 'Garcia']

,last_name,first_name,team
81,Garcia,Luis,HOU
205,Garcia,Yimi,TOR
346,Garcia,Rony,DET
409,Garcia,Jarlin,SF


In [417]:
df_probable_pitchers = pd.merge(df_probable_pitchers, misc_pitchers, on=['last_name', 'first_name', 'team'], how='left')
pitchers_df = pd.merge(pitchers_df, misc_pitchers, on=['last_name', 'first_name'], how='inner') #issue occurs on this merge
batters_df = pd.merge(batters_df, misc_batters, on=['last_name', 'first_name'], how='inner')
#Adding team to all of the players
#Joining probables as a left join so it will include all of the probables without exception -- may make one of their stats
# unavailable if they don't have enough PAs 

In [418]:
pitchers_df.loc[pitchers_df['last_name'] == 'Garcia']

,last_name,first_name,player_id,year,p_k_percent,p_bb_percent,batting_avg,xba,xslg,woba,xwoba,xobp,xiso,xbadiff,xslgdiff,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,Unnamed: 18,team
19,Garcia,Luis,472610,2022,22.7,7.6,0.246,0.292,0.378,0.255,0.319,0.346,0.086,-0.046,-0.132,91.6,5.7,2.2,NaN,HOU
20,Garcia,Luis,677651,2022,25.4,7.9,0.206,0.222,0.437,0.297,0.310,0.287,0.215,-0.016,-0.031,88.6,18.7,8.5,NaN,HOU
83,Garcia,Yimi,554340,2022,16.7,4.5,0.254,0.258,0.372,0.311,0.295,0.307,0.114,-0.004,0.035,85.1,15.9,4.0,NaN,TOR
175,Garcia,Jarlin,606424,2022,18.3,10.0,0.111,0.206,0.397,0.165,0.296,0.285,0.191,-0.095,-0.267,89.7,20.3,9.3,NaN,SF
395,Garcia,Rony,665621,2022,33.3,5.9,0.146,0.234,0.540,0.219,0.340,0.279,0.306,-0.088,-0.248,91.0,16.7,16.1,NaN,DET


In [419]:
# ^^^^ FOR PROBABLE PITCHERS MATCH IT ON TEAM AS WELL. THAT WILL AVOID THE LUIS GARCIA SITUATION

In [420]:
hello = pd.merge(df_probable_pitchers, pitchers_df)

In [421]:
df_probable_pitchers = pd.merge(df_probable_pitchers, pitchers_df, on=['team', 'last_name', 'first_name'], how='left')
#Adding stats to probable pitchers

#This is what is adding the other Luis Garcia

# Which player to pick for BTS

In [422]:
pp_xba = df_probable_pitchers[['last_name', 'first_name', 'team', 'batting_avg', 'xba', 'xwoba', 'xslg', 'p_bb_percent']]
#How can I compare this to batters df??

In [423]:
#Temporary
# pp_xba = pp_xba.drop([0])
# pp_xba = pp_xba.reset_index(drop=True)

One thing I could do is... switch the one column with the next, and then merge it with the batter's df
So for example if Oakland is playing Cleveland, I can switch the pitchers with each other. So Oakland's pitcher will be on Cleveland, and then I will have the pitcher xba to match with the team. Then I can basically set that for every member of the team, and then add that with the hitter xba and see if it meets the condition

In [424]:
pp_xba.sort_values(by='xba', ascending=False)

,last_name,first_name,team,batting_avg,xba,xwoba,xslg,p_bb_percent
3,Sanchez,Aaron,WAS,0.355,0.314,0.375,0.526,6.6
25,Flexen,Chris,SEA,0.278,0.299,0.401,0.582,8.0
17,Hudson,Dakota,STL,0.243,0.296,0.380,0.505,11.3
2,Gomber,Austin,COL,0.253,0.294,0.356,0.501,7.1
27,Irvin,Cole,OAK,0.254,0.288,0.380,0.565,6.4
12,Rogers,Trevor,MIA,0.276,0.286,0.366,0.500,9.2
22,Quintana,Jose,PIT,0.216,0.283,0.350,0.468,8.9
15,Ober,Bailey,MIN,0.237,0.281,0.362,0.528,5.9
8,Falter,Bailey,PHI,0.277,0.280,0.345,0.522,4.0
9,Carrasco,Carlos,NYM,0.257,0.279,0.322,0.445,3.7


In [442]:
batters_df.loc[batters_df['Team'] == 'PHI']

,last_name,first_name,player_id,year,b_k_percent,b_bb_percent,batting_avg,xba,xslg,woba,...,xiso,xbadiff,xslgdiff,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,Unnamed: 18,Team,pitcher_xba,xba_total
30,Segura,Jean,516416,2022,14.6,6.6,0.293,0.301,0.473,0.350,...,0.172,-0.008,-0.023,90.8,10.6,6.8,NaN,PHI,0.279,0.580
69,Gregorius,Didi,544369,2022,15.4,4.6,0.288,0.240,0.325,0.311,...,0.084,0.048,0.031,82.5,11.6,0.0,NaN,PHI,0.279,0.519
75,Herrera,Odubel,546318,2022,29.7,4.1,0.243,0.250,0.388,0.308,...,0.138,-0.007,0.055,89.2,7.1,6.1,NaN,PHI,0.279,0.529
76,Harper,Bryce,547180,2022,22.4,6.7,0.302,0.316,0.640,0.410,...,0.324,-0.014,-0.023,92.6,14.9,15.7,NaN,PHI,0.279,0.595
106,Castellanos,Nick,592206,2022,24.3,7.3,0.255,0.270,0.464,0.322,...,0.194,-0.015,-0.054,88.6,16.5,7.6,NaN,PHI,0.279,0.549
114,Realmuto,J.T.,592663,2022,25.0,7.7,0.234,0.244,0.375,0.289,...,0.132,-0.010,-0.035,86.9,12.9,3.9,NaN,PHI,0.279,0.523
135,Stubbs,Garrett,596117,2022,17.2,10.3,0.400,0.330,0.531,0.491,...,0.201,0.070,0.149,84.8,14.2,4.8,NaN,PHI,0.279,0.609
139,Quinn,Roman,596451,2022,34.2,7.9,0.171,0.187,0.283,0.206,...,0.095,-0.016,-0.083,90.9,25.7,4.5,NaN,PHI,0.279,0.466
199,Camargo,Johan,622666,2022,21.1,7.9,0.260,0.247,0.368,0.311,...,0.121,0.013,0.007,86.1,10.9,3.7,NaN,PHI,0.279,0.526
279,Hoskins,Rhys,656555,2022,26.0,10.2,0.228,0.244,0.472,0.322,...,0.227,-0.016,-0.048,90.7,20.4,10.6,NaN,PHI,0.279,0.523


In [426]:
pp_xba['Team'] = " "
#New column to house when I switch the teams with each other
#I'm doing this so I can eventually merge it onto the batters' df with the hitters' team

C:\Users\User\AppData\Local\Temp/ipykernel_13300/1964202557.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pp_xba['Team'] = " "


# Getting Pitcher Stats onto Batter DF

FOR NOW, UNTIL I CLEAN THE BELOW UP A LITTLE MORE AND HAVE IT ITERATE THROUGH AND ADJUST TO THE NUMBER OF ROWS PER DAY, JUST USE THE AMOUNT OF ROWS THAT THE LEN() SHOWS AND COMMENT OUT THE REST

In [427]:
len(pp_xba)

28

In [428]:
pp_xba['Team'][0] = pp_xba['team'][1]
pp_xba['Team'][1] = pp_xba['team'][0]
pp_xba['Team'][2] = pp_xba['team'][3]
pp_xba['Team'][3] = pp_xba['team'][2]
pp_xba['Team'][4] = pp_xba['team'][5]
pp_xba['Team'][5] = pp_xba['team'][4]
pp_xba['Team'][6] = pp_xba['team'][7]
pp_xba['Team'][7] = pp_xba['team'][6]
pp_xba['Team'][8] = pp_xba['team'][9]
pp_xba['Team'][9] = pp_xba['team'][8]
pp_xba['Team'][10] = pp_xba['team'][11]
pp_xba['Team'][11] = pp_xba['team'][10]
pp_xba['Team'][12] = pp_xba['team'][13]
pp_xba['Team'][13] = pp_xba['team'][12]
pp_xba['Team'][14] = pp_xba['team'][15]
pp_xba['Team'][15] = pp_xba['team'][14]
pp_xba['Team'][16] = pp_xba['team'][17]
pp_xba['Team'][17] = pp_xba['team'][16]
pp_xba['Team'][18] = pp_xba['team'][19]
pp_xba['Team'][19] = pp_xba['team'][18]
pp_xba['Team'][20] = pp_xba['team'][21]
pp_xba['Team'][21] = pp_xba['team'][20]
pp_xba['Team'][22] = pp_xba['team'][23]
pp_xba['Team'][23] = pp_xba['team'][22]
pp_xba['Team'][24] = pp_xba['team'][25]
pp_xba['Team'][25] = pp_xba['team'][24]
pp_xba['Team'][26] = pp_xba['team'][27]
pp_xba['Team'][27] = pp_xba['team'][26]
# pp_xba['Team'][28] = pp_xba['team'][29]
# pp_xba['Team'][29] = pp_xba['team'][28]
# pp_xba['Team'][30] = pp_xba['team'][31]
# pp_xba['Team'][31] = pp_xba['team'][30]
# pp_xba['Team'][32] = pp_xba['team'][33]
# pp_xba['Team'][33] = pp_xba['team'][32]
# pp_xba['Team'][34] = pp_xba['team'][35]
# pp_xba['Team'][35] = pp_xba['team'][34]

C:\Users\User\AppData\Local\Temp/ipykernel_13300/2797861871.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pp_xba['Team'][0] = pp_xba['team'][1]
C:\Users\User\AppData\Local\Temp/ipykernel_13300/2797861871.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pp_xba['Team'][1] = pp_xba['team'][0]
C:\Users\User\AppData\Local\Temp/ipykernel_13300/2797861871.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pp_xba['Team'][2] = pp_xba[

In [429]:
pp_xba.head()

,last_name,first_name,team,batting_avg,xba,xwoba,xslg,p_bb_percent,Team
0,Rodon,Carlos,SF,0.216,0.209,0.273,0.328,9.3,CIN
1,None,TBD,CIN,NaN,NaN,NaN,NaN,NaN,SF
2,Gomber,Austin,COL,0.253,0.294,0.356,0.501,7.1,WAS
3,Sanchez,Aaron,WAS,0.355,0.314,0.375,0.526,6.6,COL
4,Bradish,Kyle,BAL,NaN,NaN,NaN,NaN,NaN,BOS


In [430]:
# i = i + 1
# and i - 1 = i
#For odd numbers or something

In [431]:
# pp_xba.loc[0,'Team'] = pp_xba.loc[1,'Team']
# this works

In [432]:
pp_xba.head()

,last_name,first_name,team,batting_avg,xba,xwoba,xslg,p_bb_percent,Team
0,Rodon,Carlos,SF,0.216,0.209,0.273,0.328,9.3,CIN
1,None,TBD,CIN,NaN,NaN,NaN,NaN,NaN,SF
2,Gomber,Austin,COL,0.253,0.294,0.356,0.501,7.1,WAS
3,Sanchez,Aaron,WAS,0.355,0.314,0.375,0.526,6.6,COL
4,Bradish,Kyle,BAL,NaN,NaN,NaN,NaN,NaN,BOS


In [433]:
pp_xba = pp_xba.rename(columns={"batting_avg": "pitcher_batting_avg", "xba": "pitcher_xba"})
#Renaming columns for clarity

In [434]:
# pp_xba = pp_xba.drop([8])
#temporary

In [435]:
batters_df = pd.merge(batters_df, pp_xba[['Team','pitcher_xba']],on='Team', how='left')
#Getting pitcher data onto batters df

In [436]:
xba_sum = batters_df['xba'] + batters_df['pitcher_xba']
#Adding sum 

In [437]:
batters_df['xba_total'] = xba_sum
#Adding xba sum into the batters dataframe

# Which Hitters to pick for BTS

# Best Picks for Today

In [438]:
best_picks = batters_df[(batters_df['xba'] > .300) & (batters_df['pitcher_xba'] > .300) & (batters_df['xba_total'] > .650)]
best_picks

,last_name,first_name,player_id,year,b_k_percent,b_bb_percent,batting_avg,xba,xslg,woba,...,xiso,xbadiff,xslgdiff,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,Unnamed: 18,Team,pitcher_xba,xba_total
144,Daza,Yonathan,602074,2022,9.0,6.7,0.35,0.349,0.493,0.361,...,0.144,0.001,-0.093,85.9,5.7,5.5,NaN,COL,0.314,0.663


In [439]:
second_best = batters_df[(batters_df['xba'] > .290) & (batters_df['pitcher_xba'] > .290) & (batters_df['xba_total'] > .600)]
second_best

,last_name,first_name,player_id,year,b_k_percent,b_bb_percent,batting_avg,xba,xslg,woba,...,xiso,xbadiff,xslgdiff,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,Unnamed: 18,Team,pitcher_xba,xba_total
19,Brantley,Michael,488726,2022,10.9,10.9,0.283,0.326,0.544,0.347,...,0.218,-0.043,-0.130,90.3,8.9,7.0,NaN,HOU,0.299,0.625
144,Daza,Yonathan,602074,2022,9.0,6.7,0.350,0.349,0.493,0.361,...,0.144,0.001,-0.093,85.9,5.7,5.5,NaN,COL,0.314,0.663
297,Ruiz,Keibert,660688,2022,10.7,7.1,0.283,0.323,0.490,0.326,...,0.167,-0.040,-0.104,87.7,12.3,3.5,NaN,WAS,0.294,0.617
370,Alvarez,Yordan,670541,2022,18.2,13.8,0.250,0.328,0.694,0.390,...,0.366,-0.078,-0.135,95.2,11.9,16.7,NaN,HOU,0.299,0.627


In [440]:
third_best = batters_df[(batters_df['xba'] > .275) & (batters_df['pitcher_xba'] > .275) & (batters_df['xba_total'] > .580)]
third_best.sort_values('xba_total', ascending = False)

,last_name,first_name,player_id,year,b_k_percent,b_bb_percent,batting_avg,xba,xslg,woba,...,xiso,xbadiff,xslgdiff,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,Unnamed: 18,Team,pitcher_xba,xba_total
144,Daza,Yonathan,602074,2022,9.0,6.7,0.350,0.349,0.493,0.361,...,0.144,0.001,-0.093,85.9,5.7,5.5,NaN,COL,0.314,0.663
291,Olivares,Edward,658668,2022,13.2,5.3,0.371,0.355,0.568,0.402,...,0.212,0.016,-0.082,92.7,0.6,6.7,NaN,KC,0.281,0.636
370,Alvarez,Yordan,670541,2022,18.2,13.8,0.250,0.328,0.694,0.390,...,0.366,-0.078,-0.135,95.2,11.9,16.7,NaN,HOU,0.299,0.627
19,Brantley,Michael,488726,2022,10.9,10.9,0.283,0.326,0.544,0.347,...,0.218,-0.043,-0.130,90.3,8.9,7.0,NaN,HOU,0.299,0.625
297,Ruiz,Keibert,660688,2022,10.7,7.1,0.283,0.323,0.490,0.326,...,0.167,-0.040,-0.104,87.7,12.3,3.5,NaN,WAS,0.294,0.617
135,Stubbs,Garrett,596117,2022,17.2,10.3,0.400,0.330,0.531,0.491,...,0.201,0.070,0.149,84.8,14.2,4.8,NaN,PHI,0.279,0.609
156,Arcia,Orlando,606115,2022,27.3,11.4,0.316,0.319,0.618,0.367,...,0.299,-0.003,-0.171,94.7,16.7,14.8,NaN,ATL,0.286,0.605
383,Trejo,Alan,676701,2022,22.2,2.8,0.257,0.286,0.480,0.287,...,0.194,-0.029,-0.109,88.0,13.9,7.4,NaN,COL,0.314,0.600
52,Cron,C.J.,543068,2022,25.8,5.6,0.311,0.285,0.550,0.403,...,0.266,0.026,0.035,86.9,14.0,13.4,NaN,COL,0.314,0.599
328,Bohm,Alec,664761,2022,20.4,6.4,0.291,0.320,0.514,0.325,...,0.193,-0.029,-0.103,91.5,6.3,7.9,NaN,PHI,0.279,0.599


In [441]:
next_best = batters_df[(batters_df['xba_total'] > .550)]
next_best.sort_values('xba_total', ascending = False)

,last_name,first_name,player_id,year,b_k_percent,b_bb_percent,batting_avg,xba,xslg,woba,...,xiso,xbadiff,xslgdiff,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,Unnamed: 18,Team,pitcher_xba,xba_total
200,Reyes,Victor,622682,2022,15.4,7.7,0.292,0.409,0.580,0.352,...,0.171,-0.117,-0.122,90.6,4.1,5.0,NaN,DET,0.269,0.678
144,Daza,Yonathan,602074,2022,9.0,6.7,0.350,0.349,0.493,0.361,...,0.144,0.001,-0.093,85.9,5.7,5.5,NaN,COL,0.314,0.663
291,Olivares,Edward,658668,2022,13.2,5.3,0.371,0.355,0.568,0.402,...,0.212,0.016,-0.082,92.7,0.6,6.7,NaN,KC,0.281,0.636
370,Alvarez,Yordan,670541,2022,18.2,13.8,0.250,0.328,0.694,0.390,...,0.366,-0.078,-0.135,95.2,11.9,16.7,NaN,HOU,0.299,0.627
19,Brantley,Michael,488726,2022,10.9,10.9,0.283,0.326,0.544,0.347,...,0.218,-0.043,-0.130,90.3,8.9,7.0,NaN,HOU,0.299,0.625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,Diaz,Yandy,650490,2022,9.3,15.0,0.282,0.289,0.404,0.359,...,0.115,-0.007,-0.019,90.8,3.5,2.9,NaN,TB,0.264,0.553
233,Guillorme,Luis,641645,2022,14.5,10.5,0.333,0.272,0.407,0.378,...,0.135,0.061,0.032,86.8,4.9,3.5,NaN,NYM,0.280,0.552
168,Bregman,Alex,608324,2022,15.1,14.5,0.230,0.253,0.452,0.343,...,0.199,-0.023,-0.026,89.0,22.1,8.1,NaN,HOU,0.299,0.552
74,Trout,Mike,545361,2022,24.5,14.7,0.328,0.318,0.754,0.478,...,0.435,0.010,-0.061,92.5,22.2,24.7,NaN,LAA,0.233,0.551
